In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
path = '/content/drive/MyDrive/Land_Assessment/agri_data/data/'

# Preprocessing

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Land_Assessment/agri_data/agri_label.csv')

In [ ]:
df.head()

In [ ]:
df.info()

now checking all labeled images are available in the folder or not

In [ ]:

folder_images = os.listdir(path)

In [ ]:
len(folder_images)

In [ ]:
train_images = folder_images[:1200]

In [ ]:
test_images = folder_images[1200:]

In [ ]:
j=0
for i in tqdm(list(df['filename'].unique())):
    if i in folder_images:
        j+=1

print(j)

# Visualizing the Dataset

In [ ]:
 _ , axes = plt.subplots(4,4,figsize=(16,16))
for i,ax in tqdm(zip(range(16),axes.flat)):
    temp_df = df[df['filename']==folder_images[i]].reset_index(drop=True)
    temp_img = cv2.imread(path+folder_images[i])
    temp_img = cv2.cvtColor(temp_img, cv2.COLOR_BGR2RGB)
    for j in range(len(temp_df)):
        rect = cv2.rectangle(temp_img,(temp_df.loc[j,'xmin'],temp_df.loc[j,'ymin']),(temp_df.loc[j,'xmax'],temp_df.loc[j,'ymax']),(255,0,0),2,cv2.LINE_AA)
        rect_text = cv2.putText(rect,temp_df.loc[j,'class'],(temp_df.loc[j,'xmin'],temp_df.loc[j,'ymin']-8), cv2.FONT_HERSHEY_SIMPLEX,1, (255,255,0), 2, cv2.LINE_AA)
    ax.imshow(rect_text)
    plt.axis('off')


# Selective search

In [ ]:
cv2.setUseOptimized(True);
ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

In [ ]:
img = cv2.imread(path+folder_images[45])
ss.setBaseImage(img)
ss.switchToSelectiveSearchFast()
rects = ss.process()

In [ ]:
sel_rects = rects[:1200]
imOut = img.copy()
for i, rect in (enumerate(sel_rects)):
    x, y, w, h = rect
    cv2.rectangle(imOut, (x, y), (x+w, y+h), (0, 255, 0), 1, cv2.LINE_AA)
plt.figure(figsize=(8,8))
plt.imshow(imOut)

# Define some function
Now we use our defined function for getting region proposal from images. In this function we just have to add image path and dataframe which contain true label of that image in ['filename','width','height','class','xmin','ymin','xmax','ymax'] this format.

# Check for one image

In [ ]:
from region_proposals import iou_filter

In [ ]:
import torch.optim as optim

In [ ]:
#Parameter values for grid search
thresh_values = [0.5]

In [ ]:
# model = YourModel()  # Replace with your actual model instantiation
# optimizer = optim.Adam(model.parameters(), lr=0.001)
# iou_loss = IoULoss()

In [ ]:
ss,neg =iou_filter(path+folder_images[950],df,thresh=0.5)

Now visualize only region which have IoU greater than 0.5

In [ ]:
img = cv2.imread(path+folder_images[950])
for i, rect in (enumerate(ss)):
    x, y, w, h = rect[0]
    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2, cv2.LINE_AA)

plt.figure(figsize=(12,8))
plt.imshow(img)

In [ ]:
len(ss)

In [ ]:
len(neg)

Regions which has iou < 0.2 Which will use for background class training in CNN

In [ ]:
img = cv2.imread(path+folder_images[95])
for i, rect in (enumerate(neg)):
    x, y, w, h = rect

    cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2, cv2.LINE_AA)


plt.figure(figsize=(12,8))
plt.imshow(img)

# Saving all region proposal of all images

This will take arround 1.5 hour to process so if you don't want to proccess it all ready in DATA directory.

In [ ]:
# Store grid search results
grid_search_results = {}

for thresh_value in thresh_values:
    train_data = {}
    test_data = {}

    for i,img in tqdm(enumerate(train_images)):
      ss,neg =iou_filter(path + img , df , thresh=0.5)
      train_data[img] = {'region_proposal':ss,'negative_example':neg}

    for i, img in enumerate(test_images):
        ss, neg = iou_filter(path + img, df, thresh=0.5)
        test_data[img] = {'region_proposal': ss, 'negative_example': neg}

    # Store the results for this threshold value
    grid_search_results[thresh_value] = {'train_data': train_data, 'test_data': test_data}

In [ ]:
print("Grid Search Results:")
for thresh_value, result in grid_search_results.items():
    print(f"Threshold: {thresh_value}, Train Data Length: {len(result['train_data'])}, Test Data Length: {len(result['test_data'])}")

In [ ]:
# Select the best hyperparameters based on your evaluation metric (replace with your logic)
best_thresh_value = max(grid_search_results, key=lambda k: len(grid_search_results[k]['train_data']))

In [ ]:
# Use the best hyperparameters in further analysis or visualization
best_train_data = grid_search_results[best_thresh_value]['train_data']
best_test_data = grid_search_results[best_thresh_value]['test_data']

In [ ]:
# for i,img in tqdm(enumerate(train_images)):
#     ss,neg =iou_filter(path + img , df , thresh=0.5)
#     train_data[img] = {'region_proposal':ss,'negative_example':neg}

In [ ]:
# for i,img in tqdm(enumerate(test_images)):
#     ss,neg =iou_filter(path + img , df , thresh=0.5)
#     test_data[img] = {'region_proposal':ss,'negative_example':neg}

In [ ]:
#converting numpy instantant into python datatype
import json
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

In [ ]:
with open('train.json','w+') as output_file:
    json.dump(train_data,output_file,cls=MyEncoder)

In [ ]:
with open('test.json','w+') as output_file:
    json.dump(test_data,output_file,cls=MyEncoder)

checking our conversion is right or wrong

In [ ]:
with open('train.json') as train:
    train_json = json.load(train)

In [ ]:
with open('test.json') as test:
    test_json = json.load(test)

In [ ]:
train_json == train_data

In [ ]:
test_json == test_data

#Logistic Regression

In [ ]:
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score
from sklearn.model_selection import train_test_split

# Generate a random binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, random_state=42)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define and train a logistic regression model
model = LogisticRegression(random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Form a confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Calculate precision, accuracy, and recall
precision = precision_score(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Confusion Matrix:\n", cm)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)

In [ ]:
import matplotlib.pyplot as plt

labels = ['crop', 'weed']
fig, ax = plt.subplots()
im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
ax.set(xticks=np.arange(cm.shape[1]),
       yticks=np.arange(cm.shape[0]),
       xticklabels=labels, yticklabels=labels,
       xlabel='Predicted label',
       ylabel='True label')

plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax.text(j, i, format(cm[i, j], 'd'),
                ha="center", va="center",
                color="white" if cm[i, j] > cm.max() / 2. else "black")

fig.tight_layout()
plt.show()


In [ ]:
# Calculate IoU for each class
num_classes = len(labels)
iou = np.zeros(num_classes)
for c in range(num_classes):
    intersection = cm[c,c]
    union = (cm[c,:].sum() + cm[:,c].sum()) - intersection
    iou[c] = intersection / union

# Print IoU for each class
for c in range(num_classes):
    print(f"IoU for {labels[c]}: {iou[c]:.2f}")

# Plot IoU as a bar graph
plt.bar(range(num_classes), iou)
plt.xticks(range(num_classes), labels, rotation=90)
plt.xlabel('Class')
plt.ylabel('IoU')
plt.ylim((0,1))
plt.title('Intersection over Union (IoU)')
plt.show()

In [ ]:
# WIOU (Weighted IoU)
wiou = iou.mean()

# EIOU (Element-wise IoU)
eiou = iou / cm.sum(axis=1)

# CIOU (Class-wise IoU)
ciou = iou / num_classes

# Print results
print("Weighted IoU (WIOU):", wiou)
print("Element-wise IoU (EIOU):", eiou)
print("Class-wise IoU (CIOU):", ciou)


In [ ]:
# EIOU Loss (Element-wise IoU Loss)
eiou_loss = 1 - eiou

# CIOU Loss (Class-wise IoU Loss)
ciou_loss = 1 - ciou

# SIOU Loss (Sum of IoU Loss)
siou_loss = eiou_loss.sum()

# WIOU V3 Loss (Weighted IoU V3 Loss)
wiou_v3_loss = wiou - 1 / num_classes * ciou_loss

# Print results
print("Element-wise IoU Loss (EIOU Loss):", eiou_loss)
print("Class-wise IoU Loss (CIOU Loss):", ciou_loss)
print("Sum of IoU Loss (SIOU Loss):", siou_loss)
print("Weighted IoU V3 Loss (WIOU V3 Loss):", wiou_v3_loss)


#Bayesian Optimization

In [ ]:
pip install thop


In [ ]:
pip install torch

In [ ]:
from thop import profile
import torch
# Assume model is your PyTorch model
input_tensor = torch.randn(1, 3, 224, 224)
flops, params = profile(model, inputs=(input_tensor,))
print("FLOPS:", flops)


In [ ]:
import time

# Assuming rcnn_model is your RCNN model
batch_size = 16  # Adjust as needed
input_tensor = torch.randn(batch_size, 3, 224, 224)  # Adjust input size accordingly

# Measure time for inference
start_time = time.time()
for _ in range(100):  # Adjust the number of iterations
    ss(input_tensor)
end_time = time.time()

# Calculate FPS
fps = 100 / (end_time - start_time)
print("Frames per Second (FPS):", fps)


In [ ]:
import os
from skopt import BayesSearchCV
from sklearn.metrics import accuracy_score

# Define the RCNN hyperparameter search space
hyperparameter_space = {
    'learning_rate': (0.001, 0.01),
    'batch_size': (32, 128),
    'num_layers': (2, 5),
    # Add more hyperparameters as needed
}
def fitness_function(hyperparameters):
    # Initialize and train the RCNN model with the given hyperparameters
    baye_model = model(**hyperparameters)  # Instantiate your RCNN model with the hyperparameters
    baye_model.fit(train_images, **hyperparameters)  # Train the model on the training images

    # Evaluate the trained model on the validation set
    y_pred = baye_model.predict(test_images)
    accuracy = accuracy_score(test_labels, y_pred)

    return accuracy

bayes_optimizer = BayesSearchCV(
    estimator=None,
    search_spaces=hyperparameter_space,
    scoring='accuracy',
    n_iter=50,
    cv=3,
    n_jobs=-1,
    verbose=1,
    random_state=42
)

# Run the Bayesian optimization
bayes_optimizer.fit(X=None, y=None, callback=None, fit_params=None)

# Get the best hyperparameters found by Bayesian optimization
best_hyperparameters = bayes_optimizer.best_params_

# Train the RCNN model using the best hyperparameters
best_model = RCNN(**best_hyperparameters)
best_model.fit(train_images, **best_hyperparameters)  # Train the model on the entire training set

# Crow search

In [ ]:
from tensorflow.keras.models import load_model

model = load_model("my_model.h5")


In [ ]:
# Import necessary libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from cso import CrowSearchOptimizer  # Import CSO implementation (you may need to install this library)

In [ ]:

# Define the RCNN hyperparameter search space
hyperparameter_space = {
    'learning_rate': (0.001, 0.01),
    'batch_size': (32, 128),
    'num_layers': (2, 5),
    # Add more hyperparameters as needed
}

def fitness_function(hyperparameters):
    crow_model = model(**hyperparameters)
    crow_model.fit(train_images, **hyperparameters)

    y_pred = crow_model.predict(test_images)
    accuracy = accuracy_score(test_labels, y_pred)

    return accuracy

In [ ]:
# Initialize the CSO optimizer with the defined search space and fitness function
cso_optimizer = CrowSearchOptimizer(
    search_space=hyperparameter_space,
    fitness_function=fitness_function,
    num_population=20,  # Number of crows in the population
    max_iter=100,       # Maximum number of iterations
    verbose=True        # Enable verbose mode for debugging
)

# Run the CSO optimizer
best_hyperparameters = cso_optimizer.optimize()

In [ ]:
best_model = model(**best_hyperparameters)
best_model.fit(train_images, **best_hyperparameters)